### Instação das bibliotecas necessarias para execução do pipeline

In [8]:
import pandas as pd
from datetime import datetime
import re
import os

### Transformação na camada trusted da tabela de infestacao

In [9]:
# Acesso ao diretorio dos dados raw

#subir para a pasta raiz do projeto
up_dir = "../"

#acessar caminho do diretorio raw
raw_dir ="sucoslake/raw/"

#nome do arquivo que contem dos dados brutos
file = "raw_coleta_infestacao_powerapps.parquet"

data_path = up_dir+raw_dir+file
print(data_path)
df = pd.read_parquet(data_path)

../sucoslake/raw/raw_coleta_infestacao_powerapps.parquet


#### Limpeza e tratamento das colunas

In [10]:
# Transformar ano de safra em anos validos

#substitui valores invalidos e não numericos por null
df['ano_safra'] = pd.to_numeric(df['ano_safra'], errors='coerce')

#substitui valores null por 1900 e garante que o ano seja valido, ou seja, esteja dentro de um range de 1900-2100
#intervalo assumido como premissa de ano valido por mim
df['ano_safra'] = df['ano_safra'].fillna(1900)
df.loc[(df['ano_safra'] < 1900) | (df['ano_safra'] > 2100), 'ano_safra'] = 1900

In [11]:
df

,ID,ano_safra,dt_colheita,nm_fazenda,cd_talhao,cd_loc,ds_fruta,ds_sinaisPraga,pc_infestacao,nm_resp_campo,dh_ingestao
0,1,2025,45667,FAZENDA SOL,130,"-22.058,-48.181",LARANJA,NENHUM,20,JOÃO SILVA,2025-09-07 20:21:56
1,2,2025,45741,FAZENDA LUA,204,"-22.060,-48.185",MANGA,PULGÕES,55,MARIA SOUZA,2025-09-07 20:21:56
2,3,2025,45823,FAZENDA ESTRELA,58,"-22.062,-48.190",ABACAXI,MOSCA-DA-FRUTA,75,CARLOS LIMA,2025-09-07 20:21:56
3,4,2025,45874,FAZENDA SOL,120,"-22.058,-48.181",LARANJA,NENHUM,10,JOÃO SILVA,2025-09-07 20:21:56
4,5,2025,45948,FAZENDA LUA,140,"-22.060,-48.185",MANGA,PULGÕES,40,MARIA SOUZA,2025-09-07 20:21:56


In [12]:
# Transformação do tipo de dados da coluna 'dt_colheita' para date

# Ajuste para compatibilizar com data intera do exxcel
df['dt_colheita'] = pd.to_datetime(df['dt_colheita'], origin='1899-12-30', unit='D')

df['dt_colheita'] = pd.to_datetime(df['dt_colheita'], format='%d/%m/%Y')

In [13]:
# Transformar e limpar nome da fazenda

#garantir que 'nm_fazenda' é string
df['nm_fazenda'] = df['nm_fazenda'].astype(str)

#remove caracteres especiais e deixa em uppercase
df['nm_fazenda'] = df['nm_fazenda'].str.upper()
df['nm_fazenda'] = df['nm_fazenda'].str.replace(r'[^A-Z0-9 ]', '', regex=True)

In [14]:
# Transformar 'cd_talhao' em numeros validos

#substitui valores invalidos e não numericos por null
df['cd_talhao'] = pd.to_numeric(df['cd_talhao'], errors='coerce')

#tratar valores negativos
df['cd_talhao'] = df['cd_talhao'].abs()

In [15]:
# Transformar e limpar nome descrição da fruta

#garantir que 'ds_fruta' é string
df['ds_fruta'] = df['ds_fruta'].astype(str)

#remove caracteres especiais e deixa em uppercase
df['ds_fruta'] = df['ds_fruta'].str.upper()
df['ds_fruta'] = df['ds_fruta'].str.replace(r'[^A-Z0-9 ]', '', regex=True)

In [16]:
# Transformar e limpar sinais de praga

#garantir que 'ds_sinaisPraga' é string
df['ds_sinaisPraga'] = df['ds_sinaisPraga'].astype(str)

#remove caracteres especiais e deixa em uppercase
df['ds_sinaisPraga'] = df['ds_sinaisPraga'].str.upper()
df['ds_sinaisPraga'] = df['ds_sinaisPraga'].str.replace('-', ' ', regex=False)
df['ds_sinaisPraga'] = df['ds_sinaisPraga'].str.replace(r'[^A-Z0-9 ]', '', regex=True)

In [17]:
# Transformar 'pc_infestacao' em numeros validos

#substitui valores invalidos e não numericos por null
df['pc_infestacao'] = pd.to_numeric(df['pc_infestacao'], errors='coerce')

#premissa de que se o numero for negativo houve algum problema na coleta de dados
df['ano_safra'] = df['ano_safra'].fillna(-1)


In [18]:
# Transformar e limpar nome do responsavel

#garantir que 'ds_sinaisPraga' é string
df['nm_resp_campo'] = df['nm_resp_campo'].astype(str)

#remove caracteres especiais e deixa em uppercase
df['nm_resp_campo'] = df['nm_resp_campo'].str.upper()

In [19]:
df['dh_ingestao'] = pd.to_datetime(df['dh_ingestao'], format='%Y-%m-%d %H:%M:%S')

In [20]:
# Inserir datahora de inserção da trusted

df["dh_insercao_trs"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [23]:
# Acesso ao diretorio dos dados trusted

#subir para a pasta raiz do projeto
up_dir = "../"

#acessar caminho do diretorio trusted
raw_dir ="sucoslake/trusted/"

#nome do arquivo que contem dos dados brutos
file = "trusted_infestacao_powerapps.parquet"

#caminho completo
data_path = up_dir+raw_dir+file

if os.path.exists(data_path) is False: 
    df.to_parquet(data_path, engine="pyarrow", compression="snappy")
    print("Charge Full")
    
else:
    df_existente = pd.read_parquet(data_path)
    df_existente = df_existente.sort_values('dh_insercao_trs').groupby('ID', as_index=False).last()
        
    # Criar um flag de alteração comparando todos os campos exceto a chave
    key = 'ID'
    compare_cols = [
    'ano_safra',
    'dt_colheita',
    'nm_fazenda',
    'cd_talhao',
    'cd_loc',
    'ds_fruta',
    'ds_sinaisPraga',
    'pc_infestacao',
    'nm_resp_campo'
    ]

    # Merge para detectar alterações
    df_merged = df.merge(df_existente, on=key, how='left', suffixes=('_novo','_old'))
    
    # Verifica se algum campo mudou
    mask_alterado = (df_merged[[f"{c}_novo" for c in compare_cols]] != df_merged[[f"{c}_old" for c in compare_cols]].values).any(axis=1)
    
    # Seleciona apenas os registros alterados
    df_para_inserir = df[mask_alterado]
        
    # Adiciona os registros alterados ao histórico
    df_final = pd.concat([df_existente, df_para_inserir], ignore_index=True)
    
    # Salvar arquivo parquet
    df_final.to_parquet(data_path, engine="pyarrow", compression="snappy")
    print("Insert")

Insert


In [24]:
display(df_final)

,ID,ano_safra,dt_colheita,nm_fazenda,cd_talhao,cd_loc,ds_fruta,ds_sinaisPraga,pc_infestacao,nm_resp_campo,dh_ingestao,dh_insercao_trs
0,1,2025,2025-01-10,FAZENDA SOL,130,"-22.058,-48.181",LARANJA,NENHUM,20,JOÃO SILVA,2025-09-07 20:21:56,2025-09-07 20:37:21
1,2,2025,2025-03-25,FAZENDA LUA,204,"-22.060,-48.185",MANGA,PULGES,55,MARIA SOUZA,2025-09-07 20:21:56,2025-09-07 20:37:21
2,3,2025,2025-06-15,FAZENDA ESTRELA,58,"-22.062,-48.190",ABACAXI,MOSCA DA FRUTA,75,CARLOS LIMA,2025-09-07 20:21:56,2025-09-07 20:37:21
3,4,2025,2025-08-05,FAZENDA SOL,120,"-22.058,-48.181",LARANJA,NENHUM,10,JOÃO SILVA,2025-09-07 20:21:56,2025-09-07 20:37:21
4,5,2025,2025-10-18,FAZENDA LUA,140,"-22.060,-48.185",MANGA,PULGES,40,MARIA SOUZA,2025-09-07 20:21:56,2025-09-07 20:37:21
